In [1]:
# import dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import re

In [2]:
# set the chromedriver path
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser("chrome", **executable_path, headless=False)

## Scaraping the data

In [3]:
locIds = ["1133904","1151614","1140171","1150505","1147401","1139761","1147436","1154532","1138213","1152672"]
base_url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId="

city_urls = []

for locId in locIds:
     city_url = base_url + locId
     city_urls.append(city_url)

# Phoenix

In [4]:
city_urls[0]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1133904'

In [5]:
browser.visit(city_urls[0])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
phoenix_companies = []
phoenix_ratings = []
phoenix_salaries = []
phoenix_cities = []
phoenix_states = []
phoenix_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            phoenix_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            phoenix_companies.append(company.split('–')[0])
            phoenix_cities.append(company.split('–')[1])
            phoenix_states.append(company.split(',')[1])
            phoenix_ratings.append(res.find_by_css("div.logoWrap > span").text)
            phoenix_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11


In [45]:
len(phoenix_job_descriptions)

191

In [107]:
phoenix_df1 = pd.DataFrame({"Companies": phoenix_companies, 
                   "Salaries": phoenix_salaries, 
                   "Cities" : phoenix_cities,
                   "States" : phoenix_states,
                   })

In [108]:
phoenix_df2 = pd.DataFrame({"Ratings": phoenix_ratings,
                   "Job_descriptions": phoenix_job_descriptions,
                  })

In [110]:
phoenix_df1.index.name = "id" # renaming index to id

In [114]:
phoenix_df1.index = phoenix_df1.index + 1  # changing index to counting number

In [117]:
phoenix_df1.reset_index(level=0,inplace=True) # reset index as original

In [118]:
phoenix_df1.head()

,id,Companies,Salaries,Cities,States
0,1,Carvana,$114k-$161k(Glassdoor Est.),"Tempe, AZ",AZ
1,2,Banner Health,$73k-$110k(Glassdoor Est.),"Phoenix, AZ",AZ
2,3,Lyft,$82k-$120k(Glassdoor Est.),"Phoenix, AZ",AZ
3,4,USAA,$103k-$150k(Glassdoor Est.),"Phoenix, AZ",AZ
4,5,IBM,$80k-$116k(Glassdoor Est.),"Phoenix, AZ",AZ


In [119]:
phoenix_df2.index.name = "id"

In [120]:
phoenix_df2.index = phoenix_df2.index + 1

In [121]:
phoenix_df2.reset_index(level=0,inplace=True)

In [122]:
phoenix_df2.head()

,id,Ratings,Job_descriptions
0,1,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
1,2,3.4,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
2,3,4.2,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
3,4,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
4,5,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...


In [144]:
# merging to dataframe
phoenix_df = pd.merge(phoenix_df1,phoenix_df2, how="inner", on="id")

# Cleaning data

In [145]:
#removing index
del phoenix_df["id"]

In [146]:
phoenix_df.head()

,Companies,Salaries,Cities,States,Ratings,Job_descriptions
0,Carvana,$114k-$161k(Glassdoor Est.),"Tempe, AZ",AZ,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
1,Banner Health,$73k-$110k(Glassdoor Est.),"Phoenix, AZ",AZ,3.4,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
2,Lyft,$82k-$120k(Glassdoor Est.),"Phoenix, AZ",AZ,4.2,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
3,USAA,$103k-$150k(Glassdoor Est.),"Phoenix, AZ",AZ,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...
4,IBM,$80k-$116k(Glassdoor Est.),"Phoenix, AZ",AZ,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...


In [147]:
phoenix_df = phoenix_df.join(phoenix_df['Cities'].str.split(',', 1, expand=True).rename(columns={0:'CitiesA', 1:'CitiesB'}))

In [148]:
phoenix_df.head()

,Companies,Salaries,Cities,States,Ratings,Job_descriptions,CitiesA,CitiesB
0,Carvana,$114k-$161k(Glassdoor Est.),"Tempe, AZ",AZ,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Tempe,AZ
1,Banner Health,$73k-$110k(Glassdoor Est.),"Phoenix, AZ",AZ,3.4,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ
2,Lyft,$82k-$120k(Glassdoor Est.),"Phoenix, AZ",AZ,4.2,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ
3,USAA,$103k-$150k(Glassdoor Est.),"Phoenix, AZ",AZ,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ
4,IBM,$80k-$116k(Glassdoor Est.),"Phoenix, AZ",AZ,3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ


In [149]:
# removing cities and States
del phoenix_df["Cities"]
del phoenix_df["States"]

In [152]:
# renaming CitiesA to Cities
phoenix_df = phoenix_df.rename(columns={"CitiesA":"Cities"})

In [153]:
# renaming CitiesB to States
phoenix_df = phoenix_df.rename(columns={"CitiesB":"States"})

In [155]:
phoenix_df.head()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,Carvana,$114k-$161k(Glassdoor Est.),3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Tempe,AZ
1,Banner Health,$73k-$110k(Glassdoor Est.),3.4,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ
2,Lyft,$82k-$120k(Glassdoor Est.),4.2,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ
3,USAA,$103k-$150k(Glassdoor Est.),3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ
4,IBM,$80k-$116k(Glassdoor Est.),3.6,ABOUT CARVANA.\n\nRecently ranked #5 in Forbes...,Phoenix,AZ


In [156]:
# Checking if there is null values
phoenix_df.isnull()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [163]:
phoenix_df.to_csv("phoenix_data.csv", index=False)

# Portland

In [7]:
city_urls[1]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1151614'

In [8]:
browser.visit(city_urls[1])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [9]:
portland_companies = []
portland_ratings = []
portland_salaries = []
portland_cities = []
portland_states = []
portland_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            portland_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            portland_companies.append(company.split('–')[0])
            portland_cities.append(company.split('–')[1])
            portland_states.append(company.split(',')[1])
            portland_ratings.append(res.find_by_css("div.logoWrap > span").text)
            portland_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9


In [77]:
len(portland_job_descriptions)

167

In [81]:
# Creating dataframe
portland_df = pd.DataFrame({"Companies": portland_companies, 
                   "Salaries": portland_salaries, 
                   "Ratings": portland_ratings,
                   "Cities" : portland_cities,
                   "States" : portland_states,
                   "Job_descriptions": portland_job_descriptions,
                  })

# Data Cleaning

In [165]:
portland_df.head()

,Companies,Salaries,Ratings,Cities,States,Job_descriptions
0,Simple,$95k-$135k(Glassdoor Est.),2.7,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...
1,comScore,$85k-$121k(Glassdoor Est.),3.0,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...
2,First Tech Federal Credit Union,$93k-$138k(Glassdoor Est.),3.5,"Hillsboro, OR",OR,The Company:\n\nWe're a tech company that's ch...
3,ProKarma,$112k-$164k(Glassdoor Est.),3.5,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...
4,Cloudability,$123k-$173k(Glassdoor Est.),4.4,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...


In [166]:
portland_df = portland_df.join(portland_df['Cities'].str.split(',', 1, expand=True).rename(columns={0:'CitiesA', 1:'CitiesB'}))

In [169]:
portland_df.head()

,Companies,Salaries,Ratings,Cities,States,Job_descriptions,CitiesA,CitiesB
0,Simple,$95k-$135k(Glassdoor Est.),2.7,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...,Portland,OR
1,comScore,$85k-$121k(Glassdoor Est.),3.0,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...,Portland,OR
2,First Tech Federal Credit Union,$93k-$138k(Glassdoor Est.),3.5,"Hillsboro, OR",OR,The Company:\n\nWe're a tech company that's ch...,Hillsboro,OR
3,ProKarma,$112k-$164k(Glassdoor Est.),3.5,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...,Portland,OR
4,Cloudability,$123k-$173k(Glassdoor Est.),4.4,"Portland, OR",OR,The Company:\n\nWe're a tech company that's ch...,Portland,OR


In [170]:
# removing cities and States
del portland_df["Cities"]
del portland_df["States"]

In [172]:
# renaming CitiesA to Cities
portland_df = portland_df.rename(columns={"CitiesA":"Cities"})

In [173]:
# renaming CitiesB to States
portland_df = portland_df.rename(columns={"CitiesB":"States"})

In [174]:
portland_df.head()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,Simple,$95k-$135k(Glassdoor Est.),2.7,The Company:\n\nWe're a tech company that's ch...,Portland,OR
1,comScore,$85k-$121k(Glassdoor Est.),3.0,The Company:\n\nWe're a tech company that's ch...,Portland,OR
2,First Tech Federal Credit Union,$93k-$138k(Glassdoor Est.),3.5,The Company:\n\nWe're a tech company that's ch...,Hillsboro,OR
3,ProKarma,$112k-$164k(Glassdoor Est.),3.5,The Company:\n\nWe're a tech company that's ch...,Portland,OR
4,Cloudability,$123k-$173k(Glassdoor Est.),4.4,The Company:\n\nWe're a tech company that's ch...,Portland,OR


In [175]:
portland_df.isnull()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [188]:
portland_df.to_csv("portland_data.csv", index=False)

# Houston

In [10]:
city_urls[2]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1140171'

In [11]:
browser.visit(city_urls[2])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [12]:
houston_companies = []
houston_ratings = []
houston_salaries = []
houston_cities = []
houston_states = []
houston_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            houston_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            houston_companies.append(company.split('–')[0])
            houston_cities.append(company.split('–')[1])
            houston_states.append(company.split(',')[1])
            houston_ratings.append(res.find_by_css("div.logoWrap > span").text)
            houston_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11


In [90]:
len(houston_job_descriptions)

222

In [176]:
# Creating dataframe
houston_df1 = pd.DataFrame({"Companies": houston_companies, 
                   "Salaries": houston_salaries, 
                   "Cities" : houston_cities,
                   "States" : houston_states,
                   })

In [177]:
#Creating dataframe
houston_df2 = pd.DataFrame({"Ratings": houston_ratings,
                   "Job_descriptions": houston_job_descriptions,
                  })

In [178]:
houston_df1.index = houston_df1.index + 1  # changing index to counting number

In [179]:
houston_df1.index.name = "id" # renaming index to id

In [180]:
houston_df1.reset_index(level=0,inplace=True) # reset index as original

In [186]:
houston_df1.head()

,id,Companies,Salaries,Cities,States
0,1,Amazon,$112k-$159k(Glassdoor Est.),"Houston, TX",TX
1,2,Schlumberger,$103k-$153k(Glassdoor Est.),"Houston, TX",TX
2,3,N.F Smith & Associates,$73k-$104k(Glassdoor Est.),"Houston, TX",TX
3,4,Chevron,$122k-$184k(Glassdoor Est.),"Houston, TX",TX
4,5,PROS,$90k-$127k(Glassdoor Est.),"Houston, TX",TX


In [183]:
houston_df2.index = houston_df2.index + 1  # changing index to counting number

In [184]:
houston_df2.index.name = "id" # renaming index to id

In [185]:
houston_df2.reset_index(level=0,inplace=True) # reset index as original

In [187]:
houston_df2.head()

,id,Ratings,Job_descriptions
0,1,3.8,Excited by using massive amounts of data to de...
1,2,3.5,Excited by using massive amounts of data to de...
2,3,4.2,Excited by using massive amounts of data to de...
3,4,3.9,Excited by using massive amounts of data to de...
4,5,3.9,Excited by using massive amounts of data to de...


In [189]:
# merging to dataframe
houston_df = pd.merge(houston_df1,houston_df2, how="inner", on="id")

# Data Cleaning 

In [190]:
houston_df = houston_df.join(houston_df['Cities'].str.split(',', 1, expand=True).rename(columns={0:'CitiesA', 1:'CitiesB'}))

In [192]:
# removing id, cities and States
del houston_df["id"]
del houston_df["Cities"]
del houston_df["States"]

In [193]:
# renaming CitiesA to Cities
houston_df = houston_df.rename(columns={"CitiesA":"Cities"})
# renaming CitiesB to States
houston_df = houston_df.rename(columns={"CitiesB":"States"})

In [194]:
houston_df.head()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,Amazon,$112k-$159k(Glassdoor Est.),3.8,Excited by using massive amounts of data to de...,Houston,TX
1,Schlumberger,$103k-$153k(Glassdoor Est.),3.5,Excited by using massive amounts of data to de...,Houston,TX
2,N.F Smith & Associates,$73k-$104k(Glassdoor Est.),4.2,Excited by using massive amounts of data to de...,Houston,TX
3,Chevron,$122k-$184k(Glassdoor Est.),3.9,Excited by using massive amounts of data to de...,Houston,TX
4,PROS,$90k-$127k(Glassdoor Est.),3.9,Excited by using massive amounts of data to de...,Houston,TX


In [195]:
houston_df.isnull()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [196]:
houston_df.to_csv("houston_data.csv", index=False)

# Seattle

In [13]:
city_urls[3]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1150505'

In [14]:
browser.visit(city_urls[3])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
seattle_companies = []
seattle_ratings = []
seattle_salaries = []
seattle_cities = []
seattle_states = []
seattle_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            seattle_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            seattle_companies.append(company.split('–')[0])
            seattle_cities.append(company.split('–')[1])
            seattle_states.append(company.split(',')[1])
            seattle_ratings.append(res.find_by_css("div.logoWrap > span").text)
            seattle_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11
Processing Page 12
Processing Page 13
Processing Page 14
Processing Page 15
Processing Page 16
Processing Page 17
Processing Page 18
Processing Page 19
Processing Page 20
Processing Page 21
Processing Page 22
Processing Page 23
Processing Page 24
Processing Page 25
Processing Page 26
Processing Page 27
Processing Page 28
Processing Page 29
Processing Page 30
Processing Page 31


ElementDoesNotExist: no elements could be found with css "li.next"

In [204]:
seattle_df1 = pd.DataFrame({"Companies": seattle_companies, 
                   "Salaries": seattle_salaries, 
                   "Cities" : seattle_cities,
                   "States" : seattle_states,
                  })

In [205]:
seattle_df2 = pd.DataFrame({"Ratings": seattle_ratings,
                   "Job_descriptions": seattle_job_descriptions,
                  })

In [206]:
seattle_df1.index = seattle_df1.index + 1  # changing index to counting number
seattle_df1.index.name = "id" # renaming index to id
seattle_df1.reset_index(level=0,inplace=True) # reset index as original
seattle_df2.index = seattle_df2.index + 1  # changing index to counting number
seattle_df2.index.name = "id" # renaming index to id
seattle_df2.reset_index(level=0,inplace=True) # reset index as original

In [207]:
# merging to dataframe
seattle_df = pd.merge(seattle_df1,seattle_df2, how="inner", on="id")

In [208]:
seattle_df = seattle_df.join(seattle_df['Cities'].str.split(',', 1, expand=True).rename(columns={0:'CitiesA', 1:'CitiesB'}))
# removing id, cities and States
del seattle_df["id"]
del seattle_df["Cities"]
del seattle_df["States"]
# renaming CitiesA to Cities
seattle_df = seattle_df.rename(columns={"CitiesA":"Cities"})
# renaming CitiesB to States
seattle_df = seattle_df.rename(columns={"CitiesB":"States"})

In [209]:
seattle_df.head()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,Postmates Inc.,$102k-$142k(Glassdoor Est.),3.1,Postmates runs one of the largest marketplaces...,Bellevue,WA
1,Expedia,$108k-$149k(Glassdoor Est.),3.8,Postmates runs one of the largest marketplaces...,Bellevue,WA
2,EagleView Technologies,$87k-$122k(Glassdoor Est.),3.0,Postmates runs one of the largest marketplaces...,Bellevue,WA
3,"ExtraHop Networks, Inc.",$83k-$116k(Glassdoor Est.),4.0,Postmates runs one of the largest marketplaces...,Seattle,WA
4,TIBCO Software,$121k-$168k(Glassdoor Est.),3.6,Postmates runs one of the largest marketplaces...,Seattle,WA


In [210]:
seattle_df.isnull()

,Companies,Salaries,Ratings,Job_descriptions,Cities,States
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [211]:
seattle_df.to_csv("seattle_data.csv", index=False)

# San Fransisco

In [18]:
city_urls[4]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1147401'

In [19]:
browser.visit(city_urls[4])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [20]:
sanfransisco_companies = []
sanfransisco_ratings = []
sanfransisco_salaries = []
sanfransisco_cities = []
sanfransisco_states = []
sanfransisco_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            sanfransisco_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            sanfransisco_companies.append(company.split('–')[0])
            sanfransisco_cities.append(company.split('–')[1])
            sanfransisco_states.append(company.split(',')[1])
            sanfransisco_ratings.append(res.find_by_css("div.logoWrap > span").text)
            sanfransisco_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11
Processing Page 12
Processing Page 13
Processing Page 14
Processing Page 15
Processing Page 16
Processing Page 17
Processing Page 18
Processing Page 19
Processing Page 20
Processing Page 21
Processing Page 22
Processing Page 23
Processing Page 24
Processing Page 25
Processing Page 26
Processing Page 27
Processing Page 28
Processing Page 29
Processing Page 30
Processing Page 31


ElementDoesNotExist: no elements could be found with css "li.next"

In [92]:
sanfransisco_df = pd.DataFrame({"Companies": sanfransisco_companies, 
                   "Salaries": sanfransisco_salaries, 
                   "Ratings": sanfransisco_ratings,
                   "Cities" : sanfransisco_cities,
                   "States" : sanfransisco_states,
                   "Job_descriptions": sanfransisco_job_descriptions,
                  })

ValueError: arrays must all be same length

In [21]:
sanfransisco_cities

[' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' Oakland, CA',
 ' San Francisco, CA',
 ' San Mateo, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Mateo, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Bruno, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' Foster City, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San Francisco, CA',
 ' San F

# Austin

In [22]:
city_urls[5]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1139761'

In [23]:
browser.visit(city_urls[5])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [24]:
austin_companies = []
austin_ratings = []
austin_salaries = []
austin_cities = []
austin_states = []
austin_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            austin_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            austin_companies.append(company.split('–')[0])
            austin_cities.append(company.split('–')[1])
            austin_states.append(company.split(',')[1])
            austin_ratings.append(res.find_by_css("div.logoWrap > span").text)
            austin_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11
Processing Page 12
Processing Page 13


# San Jose

In [25]:
city_urls[6]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1147436'

In [26]:
browser.visit(city_urls[6])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [27]:
sanjose_companies = []
sanjose_ratings = []
sanjose_salaries = []
sanjose_cities = []
sanjose_states = []
sanjose_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            sanjose_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            sanjose_companies.append(company.split('–')[0])
            sanjose_cities.append(company.split('–')[1])
            sanjose_states.append(company.split(',')[1])
            sanjose_ratings.append(res.find_by_css("div.logoWrap > span").text)
            sanjose_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11
Processing Page 12
Processing Page 13
Processing Page 14
Processing Page 15
Processing Page 16
Processing Page 17
Processing Page 18
Processing Page 19
Processing Page 20
Processing Page 21
Processing Page 22
Processing Page 23
Processing Page 24
Processing Page 25
Processing Page 26
Processing Page 27
Processing Page 28
Processing Page 29
Processing Page 30
Processing Page 31


ElementDoesNotExist: no elements could be found with css "li.next"

# Boston

In [28]:
city_urls[7]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1154532'

In [29]:
browser.visit(city_urls[7])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [30]:
boston_companies = []
boston_ratings = []
boston_salaries = []
boston_cities = []
boston_states = []
boston_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            boston_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            boston_companies.append(company.split('–')[0])
            boston_cities.append(company.split('–')[1])
            boston_states.append(company.split(',')[1])
            boston_ratings.append(res.find_by_css("div.logoWrap > span").text)
            boston_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11
Processing Page 12
Processing Page 13
Processing Page 14
Processing Page 15
Processing Page 16
Processing Page 17
Processing Page 18
Processing Page 19
Processing Page 20
Processing Page 21
Processing Page 22
Processing Page 23
Processing Page 24
Processing Page 25
Processing Page 26
Processing Page 27
Processing Page 28
Processing Page 29
Processing Page 30
Processing Page 31


ElementDoesNotExist: no elements could be found with css "li.next"

# Washington

In [31]:
city_urls[8]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1138213'

In [32]:
browser.visit(city_urls[8])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [33]:
washington_companies = []
washington_ratings = []
washington_salaries = []
washington_cities = []
washington_states = []
washington_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            washington_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            washington_companies.append(company.split('–')[0])
            washington_cities.append(company.split('–')[1])
            washington_states.append(company.split(',')[1])
            washington_ratings.append(res.find_by_css("div.logoWrap > span").text)
            washington_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11
Processing Page 12
Processing Page 13
Processing Page 14
Processing Page 15
Processing Page 16
Processing Page 17
Processing Page 18
Processing Page 19
Processing Page 20
Processing Page 21
Processing Page 22
Processing Page 23
Processing Page 24
Processing Page 25
Processing Page 26
Processing Page 27
Processing Page 28
Processing Page 29
Processing Page 30
Processing Page 31


ElementDoesNotExist: no elements could be found with css "li.next"

# Philadelphia

In [34]:
city_urls[9]

'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=C&locId=1152672'

In [35]:
browser.visit(city_urls[9])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [36]:
philadelphia_companies = []
philadelphia_ratings = []
philadelphia_salaries = []
philadelphia_cities = []
philadelphia_states = []
philadelphia_job_descriptions = []
xpath = "//*[@id='MainCol']/div/ul/li"
def save_results():
    results = browser.find_by_xpath(xpath)
    for res in results:
        try:
            philadelphia_salaries.append(res.find_by_css("div:nth-child(3) > div:nth-child(1) > span").text)
            company = res.find_by_css("div.flexbox.empLoc > div").text
            philadelphia_companies.append(company.split('–')[0])
            philadelphia_cities.append(company.split('–')[1])
            philadelphia_states.append(company.split(',')[1])
            philadelphia_ratings.append(res.find_by_css("div.logoWrap > span").text)
            philadelphia_job_descriptions.append(browser.find_by_id("JobDescriptionContainer").text)
        except:
            pass

def close_modal():
    if len(browser.find_by_css(".xBtn")):
        browser.find_by_css(".xBtn")[0].click()

def is_last_page():
    if len(browser.find_by_css("li.page.current.last")):
        return True
    return False

def next_page():
    next_pages = browser.find_by_css("li.next")
    next_pages[0].click()

last_page = False
counter = 0
while not last_page:
    if is_last_page(): last_page = True
    close_modal()
    print(f"Processing Page {counter+1}")
    save_results()
    counter += 1
    next_page()

Processing Page 1
Processing Page 2
Processing Page 3
Processing Page 4
Processing Page 5
Processing Page 6
Processing Page 7
Processing Page 8
Processing Page 9
Processing Page 10
Processing Page 11
Processing Page 12
Processing Page 13
Processing Page 14
Processing Page 15
Processing Page 16
Processing Page 17
Processing Page 18
Processing Page 19
Processing Page 20
Processing Page 21
Processing Page 22
Processing Page 23
Processing Page 24
Processing Page 25
Processing Page 26
Processing Page 27
